In [ ]:
torchvision resnet50 v2 on imagenet
Acc@1 80.858 Acc@5 95.434

torchrun --nproc_per_node=8 train.py --model resnet50 --batch-size 128 --lr 0.5 \
--lr-scheduler cosineannealinglr --lr-warmup-epochs 5 --lr-warmup-method linear \
--auto-augment ta_wide --epochs 600 --random-erase 0.1 --weight-decay 0.00002 \
--norm-weight-decay 0.0 --label-smoothing 0.1 --mixup-alpha 0.2 --cutmix-alpha 1.0 \
--train-crop-size 176 --model-ema --val-resize-size 232 \
--ra-sampler --ra-reps=4

In [ ]:
def _list_corrupt_images(df: pd.DataFrame, archive_path: Path):
    corrupted = list()
    with zipfile.ZipFile(archive_path) as zf:
        for idx, row in tqdm(df.iterrows(), total = len(df)):
            try:
                iio.imread(zf.read(row["image_path"]), extension='.jpeg')
            except Warning as w:
                corrupted.append(idx)
                print(f"warning on idx = {idx}, [{w}]")
                continue
            except Exception as e:
                corrupted.append(idx)
                print(f"exception on idx = {idx}, [{e}]")
                continue
    return corrupted
    
def get_corrupt_images_df() -> pd.DataFrame:
    archive_path = cls.local / "staging" / "imagenet-object-localization-challenge.zip"
    df = cls.load(table = 'index', src = 'archive', subset = 'imagenet_1k')

    # TODO: setup logging and write warnings/errors to a file instead of stdout and memory
    warnings.filterwarnings('error')
    with Pool(cpu_count()-1) as pool:
        sus_idxs = pool.starmap(cls._list_corrupt_images, [(df.loc[idxs], archive_path) for idxs in np.array_split(df.index, cpu_count()-1)])
    warnings.resetwarnings()

    idxs = list() 
    for subarray in sus_idxs:
        for idx in subarray:
            idxs.append(idx)
    df = df.loc[idxs]
    df.to_hdf(archive_path.parent / "metadata.h5", key = "corrupted", mode = "a")
    return df

def filter_corrupt_images(cls, df: pd.DataFrame) -> pd.DataFrame:
    return df.drop(index=cls.load(table = 'corrupt', src = 'archive', subset = 'imagenet_1k').index)